In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from preprocess import pos_tagger
from preprocess import words_lemmatizer
from preprocess import clean_by_freq, clean_by_len, clean_by_stopwords
from collections import Counter

df = pd.read_csv('imdb.tsv', delimiter = "\\t")

# 소문자로 정규화
df['review'] = df['review'].str.lower()

# 문장 토큰화
df['sent_tokens'] = df['review'].apply(sent_tokenize)

# 품사 태깅
df['pos_tagged_tokens'] = df['sent_tokens'].apply(pos_tagger)

# 표제어 추출
df['lemmatized_tokens'] = df['pos_tagged_tokens'].apply(words_lemmatizer)

# 추가 전처리 (빈도 1 이하, 단어 길이 2 이하 단어 제거, 불용어 제거)
stopwords_set = set(stopwords.words('english'))

df['cleaned_tokens'] = df['lemmatized_tokens'].apply(lambda x: clean_by_freq(x, 1))
df['cleaned_tokens'] = df['cleaned_tokens'].apply(lambda x: clean_by_len(x, 2))
df['cleaned_tokens'] = df['cleaned_tokens'].apply(lambda x: clean_by_stopwords(x, stopwords_set))

df[['cleaned_tokens']]

# 정수 인코딩
tokens = df['cleaned_tokens'][4]

vocab = Counter(tokens)
vocab = vocab.most_common()

# 각 단어에 인덱스 부여
word_to_idx = {}
i = 0

for (word, frequency) in vocab:
    i = i + 1
    word_to_idx[word] = i

# 토큰들을 부여된 인덱스로 변경
encoded_idx = []

for token in tokens:
    idx = word_to_idx[token]
    encoded_idx.append(idx)

# 전체 코퍼스의 토큰들을 전부 합하여 단어의 등장 빈도를 계산
tokens = sum(df['cleaned_tokens'], [])

# 합쳐진 토큰 리스트로 빈도 계산, 많이 등장한 순으로 정렬하여 정수 인덱스 부여
word_to_idx = {}
i = 0
tokens = sum(df['cleaned_tokens'], [])

vocab = Counter(tokens)
vocab = vocab.most_common()

for (word, frequency) in vocab:
    i = i + 1
    word_to_idx[word] = i

# 데이터 프레임의 각 로우에 있는 토큰들을 정수 인코딩
def idx_encoder(tokens, word_to_idx):
    encoded_idx = []
    
    for token in tokens:
        idx = word_to_idx[token]
        encoded_idx.append(idx)
        
    return encoded_idx

df['integer_encoded'] = df['cleaned_tokens'].apply(lambda x: idx_encoder(x, word_to_idx))
df[['integer_encoded']]

C:\Users\parkf\AppData\Local\Temp\ipykernel_19668\2903358726.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\parkf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\parkf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\parkf\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is alread

,integer_encoded
0,"[8, 11, 2, 54, 8, 16, 5, 1, 12, 54, 8, 16, 5, ..."
1,"[2, 2]"
2,"[55, 56, 19, 57, 58, 59, 57, 60, 61, 25, 62, 6..."
3,"[2, 2, 30, 69, 3, 30, 31, 3, 69, 70, 71, 30, 7..."
4,"[72, 1, 5, 17, 1, 1, 8, 1, 73, 74, 1, 1, 74, 1..."
5,"[35, 36, 78, 78, 35, 36, 79, 79, 35, 36]"
6,"[80, 1, 3, 21, 81, 82, 21, 21, 80, 3, 82, 83, ..."
7,"[85, 86, 37, 38, 87, 39, 86, 88, 40, 89, 41, 9..."
8,"[120, 1, 1, 121, 52, 10, 121, 122, 53, 1, 52, ..."
9,"[123, 123]"


In [2]:
# 제로 패딩
max_len = max(len(item) for item in df['integer_encoded'])

print('토큰의 최대 개수:', max_len)

토큰의 최대 개수: 200


In [3]:
# 다른 코퍼스들의 길이가 200이 되도록 0 채워넣기
for tokens in df['integer_encoded']:
    while len(tokens) < max_len:
        tokens.append(0)

df[['integer_encoded']]

,integer_encoded
0,"[8, 11, 2, 54, 8, 16, 5, 1, 12, 54, 8, 16, 5, ..."
1,"[2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[55, 56, 19, 57, 58, 59, 57, 60, 61, 25, 62, 6..."
3,"[2, 2, 30, 69, 3, 30, 31, 3, 69, 70, 71, 30, 7..."
4,"[72, 1, 5, 17, 1, 1, 8, 1, 73, 74, 1, 1, 74, 1..."
5,"[35, 36, 78, 78, 35, 36, 79, 79, 35, 36, 0, 0,..."
6,"[80, 1, 3, 21, 81, 82, 21, 21, 80, 3, 82, 83, ..."
7,"[85, 86, 37, 38, 87, 39, 86, 88, 40, 89, 41, 9..."
8,"[120, 1, 1, 121, 52, 10, 121, 122, 53, 1, 52, ..."
9,"[123, 123, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
